In [1]:
# Import the necessary modules
from google.cloud import bigquery

from classes.BigQueryIO import BigQueryIO
from classes.ConfigManager import ConfigManager

#Setup configuration
config = ConfigManager(
    yaml_filepath='C:/Users/Admin/OneDrive/Desktop/_work/__repos (unpublished)/_____CONFIG/google-analytics-insight-generation/config',
    yaml_filename='config.yaml'
    )

#Create Clients
bq_client = bigquery.Client(project = config.bq_project_id)
bq_io = BigQueryIO(bq_client)

2024-01-24 10:07:38,951 - ConfigManagerClass - DEBUG - Name: load_json_runtime_table_ids - Line: 55 - Loaded runtime_table_ids_json: {'table_ids': ['_sessions', '_users', '_items', '_transaction_items', '_transactions', '__prod_users_details']}


In [2]:
# Set to query using .sql or create/replace table on BigQuery
query_vs_create = 'create' #'query'

# Set table_id to create/query BigQuery (use table_id explicitly, '', 'all', or None)
runtime_table_id = '_transaction_items' # _transaction_items

In [3]:
# Get ready for looping!
print(f"runtime_table_ids_json (type: {type(config.runtime_table_ids_json)}):{config.runtime_table_ids_json}")
print(f"runtime_table_ids_json['table_ids'] (type: {type(config.runtime_table_ids_json['table_ids'])}): {config.runtime_table_ids_json['table_ids']}")

#If custom table ID is set
if runtime_table_id != 'all' and runtime_table_id != '' and runtime_table_id is not None:
    try:
        if runtime_table_id not in config.runtime_table_ids_json['table_ids']:
            raise ValueError(f"Error: runtime_table_id: '{runtime_table_id}' not found in runtime_table_ids_json")
        else:
            table_ids = [runtime_table_id]
            print(f"table_ids: '{table_ids}'")
    except KeyError:
        print(f"Error: Table ID '{runtime_table_id}' not found in configuration.")
    
    except ValueError as e:
        print(e)

#If set to 'all'
else:
    table_ids = config.runtime_table_ids_json['table_ids']
    print(f"table_ids: {table_ids}")

runtime_table_ids_json (type: <class 'dict'>):{'table_ids': ['_sessions', '_users', '_items', '_transaction_items', '_transactions', '__prod_users_details']}
runtime_table_ids_json['table_ids'] (type: <class 'list'>): ['_sessions', '_users', '_items', '_transaction_items', '_transactions', '__prod_users_details']
table_ids: '['_transaction_items']'


In [5]:
# loop 
for table_id in table_ids:
    try:
        # Get self.bq_table_config json
        print(f"this is the config.bq_table_config: {config.bq_table_config}")
        bq_table_config = config.bq_table_config[table_id]
        print(f"this is the bq_table_config: {config.bq_table_config}")
        
        # Generate the query using the SQL file from bq_table_config['query_path']
        query = bq_io.execute_query_from_filepath(
            dataset_id = bq_table_config['dataset_id'],
            table_id = table_id,
            sql_file_path= bq_table_config['query_path'],
            query_vs_create=query_vs_create
        )

        if query_vs_create == 'create':
            print(f"Finished creating table_id: '{table_id}'. No preview available.")
        else:
            print(f"Preview of query:")
            print(query.head(5))
        print("\n")

    except KeyError as e:
        print(f"KeyError encountered for table_id '{table_id}': {e}. This may be due to an invalid table_id in your config.")
    except Exception as e:
        print(f"An unexpected error occurred for table_id '{table_id}': {e}")

2024-01-24 10:09:07,963 - bigquery_io - INFO - Name: _generate_bq_query_from_file - Line: 58 - The BigQuery table creation query for [_transaction_items] was generated from a local file:
2024-01-24 10:09:07,963 - bigquery_io - INFO - Name: _generate_bq_query_from_file - Line: 62 - 



this is the config.bq_table_config: {'_transaction_items': {'execution_order_id': 1, 'dataset_id': 'key-utility-407314.eh_ga4_stg', 'query_path': 'C:/_repos/google-analytics-insight-generation/sql/_transaction_items.sql', 'schema': [{'name': 'name', 'type': 'STRING', 'mode': 'REQUIRED'}, {'name': 'age', 'type': 'INTEGER', 'mode': 'REQUIRED'}]}, '_items': {'execution_order_id': 1, 'dataset_id': 'key-utility-407314.eh_ga4_stg', 'query_path': 'C:/_repos/google-analytics-insight-generation/sql/_items.sql', 'schema': [{'name': 'name', 'type': 'STRING', 'mode': 'REQUIRED'}, {'name': 'age', 'type': 'INTEGER', 'mode': 'REQUIRED'}]}, '_transactions': {'execution_order_id': 1, 'dataset_id': 'key-utility-407314.eh_ga4_stg', 'query_path': 'C:/_repos/google-analytics-insight-generation/sql/_transactions.sql', 'schema': [{'name': 'name', 'type': 'STRING', 'mode': 'REQUIRED'}, {'name': 'age', 'type': 'INTEGER', 'mode': 'REQUIRED'}]}, '_users': {'execution_order_id': 1, 'dataset_id': 'key-utility-4073